In [22]:
# -*- coding: utf-8 -*-
import json
import collections
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import math
import csv
from alchemyapi import AlchemyAPI

def open_json_review_files(cruiseLines):
    """
    creates dictionary with one or more cruise lines review json files
    """
    lineDb= {}
    commentDb = {}
    totcount = 0
    for line in cruiseLines:
        count = 0
        with open('data/'+line+'.json', 'rb') as fp:
            lineDb[line] = json.load(fp)
            commentDb.update(lineDb)
            for element in commentDb[line]:
                commentDb[line][element]["line"]=line
                count = count + 1 
        totcount=totcount+count
        print "processed", line, len(commentDb[line]),"reviews"
    print 'total processed', totcount, 'reviews'
    return commentDb

def cleanRatings(commentDb,cruiseLines):
    """
    cleans up and aggregates ratings data
    """
    ratingAggregations={"6":"good","5":"good","4":"medium","3":"medium","2":"bad","1":"bad","no rating":"no rating"}
    count = 0 
    for line in cruiseLines:
        for element in commentDb[line]:
            if len(commentDb[line][element]["kind"]) <5:
                   commentDb[line][element]["kind"]="not specified"
            if not commentDb[line][element]["rating"].isdigit():
                    commentDb[line][element]["rating"]="no rating" 
            if len(commentDb[line][element]["ship"]) <5:
                    commentDb[line][element]["ship"]="not available"                     
            commentDb[line][element]["aggregatedRating"]=ratingAggregations[commentDb[line][element]["rating"]] 
    return commentDb

def checkDailyQuotaAndRunAlchemy(commentDb,cruiseLines,calls):
    returned_data={}
    alchemyapi = AlchemyAPI()
    for call in calls:
        fileName='data/Alchemy_response_'+call+'.json'
        with open(fileName, 'rb') as fp:
                returned_data[call] = json.load(fp)
        test="test if finished Alchemy daily quota"
        response = alchemyapi.keywords('text', test, {'sentiment': 0})
        if response['status'] == 'OK':
            returned_data=runAlchemyApi(cruiseLines,commentDb,returned_data,call,alchemyapi)
        else:
            print 'Error in',call,' extraction call: ', response['statusInfo']
    return returned_data

def merge_old_and_new_alchemy_files(returned_data,calls,commentDb):
    returned_data_old={}
    elements_to_drop=[]
    for call in calls:
        for element in returned_data[call]:
            if returned_data[call][element]['rating']=='no rating':
                elements_to_drop.append(element)
        for key in elements_to_drop:
                returned_data[call].pop(key, None)       
        fileName='data/Alchemy_response_'+call+'_old.json'
        with open(fileName, 'rb') as fp:
                returned_data_old[call] = json.load(fp)                               
        for element in returned_data_old[call]:
            if element not in returned_data[call]:
                returned_data[call][element]=returned_data_old[call][element] 
    return returned_data

def callAPI(text,alchemyapi,call,count,returned_data,commentDb,element,line,daily_alchemy_finished):
    api_call={'keywords':alchemyapi.keywords('text', text, {'sentiment': 1}),
             'relations':alchemyapi.relations('text', text, {'sentiment': 1, 'entities':1,'keywords':0}),
              'entities':alchemyapi.entities('text', text, {'sentiment': 1,'knowledgeGraph':1}),
              'concepts':alchemyapi.entities('text', text, {'showSourceText':1,'knowledgeGraph':1})
             }
    response = api_call[call]
    if response['status'] == 'OK':
        returned_data[call]=load_element_data(response,returned_data[call],commentDb,element,line)            
        count=count+1       
    else:
        print 'Error in keyword extaction call: ', response['statusInfo']
        daily_alchemy_finished="True"
    return returned_data, count,daily_alchemy_finished
    

def runAlchemyApi(cruiseLines, commentDb,returned_data,call,alchemyapi):
    count=0
    daily_alchemy_finished=False
    check=0
    low_ratings={"1","2","5","6"}    
    conta=0
    for line in cruiseLines:
        print 'getting Alchemy',call,'data. We already have keywords of',len(returned_data[call]),"reviews for", line
        for element in commentDb[line]:
            text=commentDb[line][element]["comment"]
            if daily_alchemy_finished or element in returned_data or len(text) <100 or commentDb[line][element]['rating'] == "no rating":
                pass
            else:
                if call!="keywords" or commentDb[line][element]['rating'] in low_ratings:
                    returned_data,count,daily_alchemy_finished=callAPI(text,alchemyapi,call,count,returned_data,commentDb,element,line,daily_alchemy_finished)
    
    
    print 'finished getting',call,'data from Alchemy for', count,"reviews"        
    print 'in total we have got',call,'data for', len(returned_data[call]),"reviews"
    save_dictionary(returned_data[call],call) 
    return returned_data

def load_element_data(response,returned_data,commentDb,element,line):   
    returned_data[element]={}
    returned_data[element]=response
    returned_data[element]["rating"]=commentDb[line][element]['rating']
    returned_data[element]['ship']=commentDb[line][element]['ship']
    returned_data[element]['sail_date']=commentDb[line][element]['sail Date']
    returned_data[element]['line']=line
    return returned_data

def save_dictionary(returned_dictionary,call):
    with open('data/Alchemy_response_'+call+'.json', 'wb') as fp:
        json.dump(returned_dictionary, fp) 
        
def check_what_lines_we_are_processing(df):
    series = df['Line'].value_counts()
    print "we are processing",series.head(), '\n'
                     
def make_keywords_csv_alchemy(returned_keywords,commentDb):
    """
    generates the csv file that powers the keyword dashboard
    """
    call="keywords"
    df=clean_dictionary_keys(returned_keywords,call,commentDb)
    #check_what_lines_we_are_processing(df)    
    patterns = [(r'[^A-Za-z0-9 ]+',''),(r' +',' ')]
    regex_columns=["Word"]
    df=clean_with_regex(regex_columns,patterns,df)    
    columns=["Word"]
    make_lowercase(columns,df)
    mask=df.Word.str.len() >= 2
    df=delete_useless_rows(mask,df)   
    mask=df.Rating!="no rating"
    df=delete_useless_rows(mask,df)  
    to_float=["relevance","sentiment_score","Rating"]
    df=convert_to_float(to_float, df)
    minimum_scores={"standard":0.50,"high":0.6,"very_high":0.7}
    for hypothesis in minimum_scores:
        filter_data_and_generate_hypothesis(df,hypothesis,minimum_scores)        
        

def make_relations_csv_alchemy(returned_relations,commentDb):
    """
    generates the csv file that powers the relations dashboard
    """
    call="relations"
    df=clean_dictionary_keys(returned_relations,call,commentDb)
    #check_what_lines_we_are_processing(df)
    patterns = [(r'[^A-Za-z0-9%\' ]+',''),(r' +',' ')]
    text_columns=["sbjText","actText", "objText","location.text"]
    df=clean_with_regex(text_columns,patterns,df)
    make_lowercase(text_columns,df)
    to_float=["sbjSentScore","objSentScore","object.sentimentFromSubject.score","location.sentiment.score"]
    df=convert_to_float(to_float, df)    
    save_csv(df,call)   

    
    
def make_entities_csv_alchemy(returned_relations,commentDb):
    """
    generates the csv file that powers the relations dashboard
    """
    call="entities"
    df=clean_dictionary_keys(returned_relations,call,commentDb)
    patterns = [(r'[^A-Za-z0-9%\' ]+',''),(r' +',' ')]
    text_columns=["text"]
    to_drop=["disambiguated.census","disambiguated.ciaFactbook","disambiguated.crunchbase","disambiguated.dbpedia",
             "disambiguated.freebase","disambiguated.geo","disambiguated.geonames","disambiguated.musicBrainz",
             "disambiguated.name","disambiguated.opencyc","disambiguated.subType","disambiguated.website",
             "disambiguated.yago","sentiment.mixed","count"]
    df=drop_columns(to_drop,df)
    mask=df["sentiment.type"]!="neutral"
    df=delete_useless_rows(mask,df)  
    all_types=["City","Organization","Continent","Country","HealthCondition","Person","Region","FieldTerminology",
               "TelevisionStation","Company","StateOrCounty","Holiday","Degree","JobTitle","Facility","GeographicFeature",
               "Movie","NaturalDisaster","Technology","PrintMedia","Drug","Sport",
               "OperatingSystem","Product","TelevisionShow","EntertainmentAward","MusicGroup","ProfessionalDegree"]
    interesting_types=["Organization","HealthCondition","JobTitle","ProfessionalDegree"]
    df=df[df["type"].isin(interesting_types)]    
    df=clean_with_regex(text_columns,patterns,df)
    make_lowercase(text_columns,df)
    to_float=[]
    df=convert_to_float(to_float, df)    
    save_csv(df,call)  
    
def make_concepts_csv_alchemy(returned_concepts,commentDb):
    pass
    
    
def clean_dictionary_keys(returned_dictionary,call,commentDb):
    df=pd.DataFrame()
    count_keywords=0
    count_relations=0
    conta=0
    keys_to_drop=["language","status","usage","totalTransactions","url"]
    to_rename=  {"keywords":{"text":"Word",'relevance_x': 'relevance','sentiment.score': 'sentiment_score',
        "sentiment":"dictionary",'sentiment.type': 'Sentiment',"rating":"Rating","ship":"Ship","line":"Line"},
        "relations":{"action.lemmatized":"actLemma",
                     "action.text":"actText",
                     "action.verb.negated":"actVerbNeg",
                     "action.verb.text":"actVerbText",
                     "object.sentiment.score":"objSentScore",
                     "object.sentiment.type":"objSentType",
                     "object.text":"objText",
                     "subject.sentiment.score":"sbjSentScore",
                     "subject.sentiment.type":"sbjSentType",
                     "subject.text":"sbjText"           
                     },
          "entities":{}}
    for review in returned_dictionary:
        if returned_dictionary[review]["language"]!="english":
            print review, "review seems not to be in English, but in", returned_dictionary["language"]
        else:
            for key in keys_to_drop:
                returned_dictionary[review].pop(key, None)
        if "line" in returned_dictionary[review]:        
            line=returned_dictionary[review]["line"]                
            df=flatten_dictionary(returned_dictionary,review,call,df,commentDb,line)
    print df.info()
    df.rename(columns=to_rename[call], inplace=True)
    return df
                
        
def clean_with_regex(regex_columns,patterns,df):
    for column in regex_columns:
        for pattern in patterns:
            df[column].replace(pattern[0],pattern[1], regex = True, inplace=True)
    return df

def filter_data_and_generate_hypothesis(df,hypothesis,minimum_scores):        
    min_relevance=minimum_scores[hypothesis]
    min_sentiment=minimum_scores[hypothesis]
    mask=df.relevance>=min_relevance
    df=delete_useless_rows(mask,df)
    df=change_unsure_to_neutral(min_sentiment,df)    
    mask=df.Sentiment!="neutral"
    df=delete_useless_rows(mask,df)
    df=aggregate_ratings(df)
    mask=df.Rating!="medium"
    df=delete_useless_rows(mask,df) 
    df=reformat_dates(df)
    to_drop=["sentiment_score","relevance","sail_date","date","dictionary","relevance_y","sentiment.mixed"]
    df=drop_columns(to_drop,df)    
    to_add={"count":1}
    df=add_columns(to_add,df)
    df=make_pivot(df)
    df["Total"]=df["Positive"]+df["Negative"]
    to_rename={"rating":"Rating"}
    df.rename(columns=to_rename, inplace=True)
    save_csv(df,hypothesis)

def flatten_dictionary(returned_dictionary,review,call,df,commentDb,line): 
    to_split=["location","object","subject"]
    if call == "keywords":
        first_level=json_normalize(returned_dictionary[review],call,['rating','sail_date',"ship","line"])
        second_level=json_normalize(returned_dictionary[review][call])
        together=pd.merge(first_level, second_level, on='text', how='outer')
        df=pd.concat([df, together])
    else:
        if call == "relations":
            for element in returned_dictionary[review][call]:
                for my_column in to_split:
                    if my_column in element:
                        if "entities" in element[my_column]:
                            first_name=my_column+'_ent_text'
                            second_name=my_column+'_ent_type'
                            element[first_name]=element[my_column]["entities"][0]["text"]
                            element[second_name]=element[my_column]["entities"][0]["type"]
                            del element[my_column]                  
        if review in commentDb[line]:
            rating=commentDb[line][review]["rating"]
            for element in returned_dictionary[review][call]:
                second_level=json_normalize(element)
                second_level['review']=review
                second_level['rating']=rating                
                df=pd.concat([df,second_level])
    return df 


def open_csv(suffix):
    df= pd.DataFrame(pd.read_csv('data/alchemy_ratings_'+suffix+'.csv'))
    return df

def drop_columns(to_drop,df):    
    for column in to_drop:
        df.drop(column, axis=1, inplace=True)
    return df

def fill_na_columns(df,lista,fill_with):
    for column in lista:
        df[column].fillna(fill_with,inplace=True)
        df["objSentScore"].fillna(0,inplace=True)   
    return df

def reorder_column(df,new_list):
    varlist =[w for w in df.columns if w not in new_list]
    df = df[new_list+varlist]
    return df 


def delete_useless_rows(mask,df):
    df=df[mask]
    return df

def reformat_dates(df):
    df["date"]=pd.to_datetime(df.sail_date, format= "%B %Y", coerce= True)
    df["Year"]=pd.DatetimeIndex(df.date).year
    return df

def add_columns(to_add,df):                
    for column in to_add:
        df[column]=to_add[column]
    return df

def make_lowercase(columns,df):
    for column in columns:
        df[column]=df[column].str.lower()
    return df

def convert_to_float(to_float, df):
    for column in to_float:
        df[column]=df[column].astype(float).fillna(0.0)
    return df

def change_unsure_to_neutral(min_sentiment,df):
    df["Sentiment"][(df["sentiment_score"] <= min_sentiment) & (df["sentiment_score"] >= -min_sentiment)]="neutral"
    return df

def make_pivot(df):
    df=df.reset_index()  
    dataDims=["Word","Rating","Year","Ship","Line"]
    valueDims=["count"]
    columnDims=["Sentiment"]
    labels=['Negative','Positive']
    pivotValues=preparePivotValues(columnDims,df)   
    df=pd.pivot_table(df,values=valueDims,index=dataDims,columns=columnDims,aggfunc='sum').fillna(0)
    result={}
    for valueDim in valueDims:
        result[valueDim]= df[valueDim]
        print 'processing ', valueDim, ' valuedimension'        
        result[valueDim] =renamePivotColumns(result[valueDim],labels,pivotValues[0]) 
        result[valueDim]=result[valueDim].reset_index()
        print 'processing pivot with ', len(columnDims) ,' pivot column dimensions and ', len(valueDims),' value column dimensions.....'        
    if len(columnDims) == 1 and len(valueDims) == 1:
        df=result[valueDim]          
    df.fillna(0, inplace=True)
    return df

    
def renamePivotColumns(result,labels,dimensions):
    count=0    
    print "CHECK THAT RENAMING IS CORRECT ==>" 
    for dimension in dimensions:
        print "renamed ", dimension, " as ", labels[count]                
        result.rename(columns={dimension:labels[count]}, inplace=True) 
        count=count+1
    print "\n "    
    return result    
    
    
def preparePivotValues(columnDims,df):
    pivotValues={}
    count = 0    
    for columnDim in columnDims:
        pivotValues[count]=df[columnDims[count]].unique()
        pivotValues[count].sort()
        if (len(pivotValues[count]) != 2):
            print 'More than two pivot values in a column. POSSIBLE ERROR IN DATA ==> ', pivotValues[count] 
        count = count + 1
    print "these are the pivot values ",pivotValues
    return pivotValues    


def prepare_keywords(sum_field,rating,suffix,top_number,line):
    df=open_csv(suffix)
    mask=df.Line==line
    df=delete_useless_rows(mask,df)
    mask=df.Rating==rating
    df=delete_useless_rows(mask,df)
    df=get_top_words(df,sum_field,top_number)
    return df

def get_top_words(df,sum_field,top_number):
    df = df.groupby('Word')
    df=df[sum_field].sum().order(ascending=False)
    df=df.head(top_number).copy()
    return df   

def save_csv(df,hypothesis):
    fileName='data/alchemy_ratings_'+hypothesis+'.csv'
    print fileName  
    df.to_csv(fileName,encoding='utf-8')

def blank_out_short_sentences(df,blank_out_rules):
    for rule in blank_out_rules:
        df.loc[df[rule[0]].str.len() <=rule[1], rule[0]] = ""
    return df    

def make_top_dataframes(top_number,line):
    """
    spits out top keywords used in negative reviews
    """
    suffix="standard"
    top_neg_bad=prepare_keywords("Negative","bad",suffix,top_number,line)
    top_neg_bad=top_neg_bad.reset_index()
    return top_neg_bad

def prepare_relations():
    """
    gets cvs output from API relations call and 
    cleans it up for dc.js
    """
    suffix="relations"
    df=open_csv(suffix)
    df=df.drop_duplicates()
    #blank_out_rules=(("objText",3),("sbjText",3))
    #df=blank_out_short_sentences(df,blank_out_rules)    
    to_rename={"object.sentimentFromSubject.score":"objSentFromSbjScore",
               "object.sentimentFromSubject.type":"objSentFromSbjType",
               "action.verb.tense":"actVerbTense",
               "location.sentiment.score":"locSentScore",
               "location.sentiment.type":"locSentType",
               "location.text":"locText"
               }
    df.rename(columns=to_rename, inplace=True)
    fill_na_str=["objText","sbjText","actVerbText","locText"]
    fill_with=""
    df=fill_na_columns(df,fill_na_str,fill_with)
    fill_na_int=["actVerbNeg","objSentScore","objSentFromSbjScore","sbjSentScore","locSentScore"]
    fill_with=0
    df=fill_na_columns(df,fill_na_int,fill_with) 
    fill_na_sent=["objSentType","objSentFromSbjType","sbjSentType","locSentType"]
    fill_with="neutral"
    df=fill_na_columns(df,fill_na_sent,fill_with)
    to_rename={"rating":"Rating"}
    df.rename(columns=to_rename, inplace=True)
    df=aggregate_ratings(df)
    to_rename={"Rating":"rating"}
    df.rename(columns=to_rename, inplace=True)
    new_order=["review","rating","sbjText","actText","actVerbNeg","objText","locText","sbjSentType",
               "objSentType","objSentFromSbjType","locSentType","sbjSentScore","objSentScore",
               "objSentFromSbjScore","actVerbText","locSentScore","actLemma","actVerbTense"]    
    df = reorder_column(df,new_order)
    to_drop=["Unnamed: 0","actLemma","actVerbText","locSentScore","actVerbTense",
             "locSentType","locText","review","objSentFromSbjScore","objSentFromSbjType"]
    to_drop=["Unnamed: 0"]
    df=drop_columns(to_drop,df)
    return df

     
def aggregate_ratings(df):
    conditions=[((df["Rating"] >= 5) & (df["Rating"] <= 6),"good"),
                ((df["Rating"] >= 3) & (df["Rating"] <= 4),"medium"),
                ((df["Rating"] >= 1) & (df["Rating"] <= 2),"bad")]
    for condition in conditions:
        df["Rating"][condition[0]]=condition[1]
    return df 

def find_keywords_in_text(df, top_bad):
    """
    Finds phrases that contains keywords and aggregates then according to similar issues. 
    ++> Must manually fill in 'interest_and_aggregation' dictionary with interesting and non interesting top keywords words.
    
    """
    count=0
    with open('data/interest_and_aggregation.json', 'rb') as fp:
            interest_and_aggregation = json.load(fp)
    text_columns=["objText","sbjText","actVerbText"]
    for word in top_bad["Word"]:
        count=count + 1
        if word in interest_and_aggregation:
            print count, word, interest_and_aggregation[word]["interesting"], interest_and_aggregation[word]["aggregation"]   
        else:
            print "===>",count, word, 'not anywhere'      
    for word in top_bad["Word"]:
        if interest_and_aggregation[word]["interesting"]==1:
            for column in text_columns:
                df.loc[df[column].str.contains(word), "foundKeyword"] = word
                df.loc[df[column].str.contains(word), "aggKeyword"] = interest_and_aggregation[word]["aggregation"]
    df=df[pd.notnull(df["foundKeyword"])]
    df["count"]=1
    save_csv(df,"relations_cleaned")
    mask=df.actVerbNeg!=1
    df=delete_useless_rows(mask,df)
    mask=df.aggKeyword!=None
    df=delete_useless_rows(mask,df)
    df["message"]=df["sbjText"]+" "+df["actText"]+" "+df["objText"]+" "+df["locText"]
    df=df.reset_index()
    to_drop=["review","locText","actVerbNeg","actLemma","actVerbTense","actText", "sbjText","objText",
             "foundKeyword","locSentType","locSentScore","objSentFromSbjType","objSentFromSbjScore"]
    df=drop_columns(to_drop,df)
    new_order=["rating","aggKeyword","message","actVerbText","sbjSentType",
               "objSentType","sbjSentScore","objSentScore","count"]    
    df = reorder_column(df,new_order)
    to_rename={"rating":"Rating","aggKeyword":"AggKeyword","message":"Message","actVerbText":"ActVerbText",
                "sbjSentType":"SbjSentType","objSentType":"ObjSentType","sbjSentScore":"SbjSentScore",
                "objSentScore":"ObjSentScore","count":"Count"}
    df.rename(columns=to_rename, inplace=True)
    condition=[(df["SbjSentScore"] >=-.6) & (df["SbjSentScore"] <= .6),0]
    df["SbjSentScore"][condition[0]]=condition[1]
    df.ix[df.SbjSentScore == 0, 'SbjSentType'] = "neutral"
    condition=[(df["ObjSentScore"] >=-.7) & (df["ObjSentScore"] <= .7),0]
    df["ObjSentScore"][condition[0]]=condition[1]
    df.ix[df.ObjSentScore == 0, 'ObjSentType'] = "neutral"
    to_drop=["ObjSentScore","SbjSentScore","index"]
    df=drop_columns(to_drop,df)
    df=df[(df['SbjSentType'] != 'neutral') | (df['ObjSentType']!= 'neutral')]
    save_csv(df,"relations_small")
    return df



def show_file(returned_data, number):
    count = 0
    for element in returned_data["relations"]:
        if count ==number:
            my_json=returned_data["relations"][element]
            print json.dumps(my_json, indent=4, sort_keys=True)
        count = count + 1



cruiseLines=["Msc","Costa"]
def main():
    calls=['keywords','relations','entities','concepts']
    calls=['relations']
    #commentDb=open_json_review_files(cruiseLines)
    #commentDb=cleanRatings(commentDb,cruiseLines)
    #returned_data=checkDailyQuotaAndRunAlchemy(commentDb,cruiseLines,calls)
    #returned_data=merge_old_and_new_alchemy_files(returned_data,calls,commentDb)
    #number = 3
    #show_file(returned_data,number)
        
    
    #make_keywords_csv_alchemy(returned_data["keywords"],commentDb)
    #make_relations_csv_alchemy(returned_data["relations"],commentDb)
    #make_entities_csv_alchemy(returned_data["entities"],commentDb)
    #make_concepts_csv_alchemy(returned_data["concepts"],commentDb)
    top_keywords=50
    df=prepare_relations()
    top_bad=make_top_dataframes(top_keywords,cruiseLines[0])
    df=find_keywords_in_text(df, top_bad)    
    print df.info()
    
main()




C:\Users\fabio\Anaconda\lib\site-packages\IPython\kernel\__main__.py:472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\fabio\Anaconda\lib\site-packages\IPython\kernel\__main__.py:497: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1 food 1 food
2 cruise line 1 overall
3 people 1 people
4 msc 1 overall
5 dining room 1 dining
6 worst cruise 0 overall
7 cruise 1 overall
8 ship 1 ship
9 cruise ship 1 ship
10 time 1 time
11 buffet 1 food
12 room service 1 food
13 board 1 ship
14 poor quality 1 service
15 msc cruise 1 overall
16 customer service 1 service
17 bar service 1 dining
18 poor service 1 service
19 passengers 1 people
20 cruise lines 1 overall
21 cruise director 1 service
22 poor food 1 food
23 public areas 1 ship
24 credit card 0 None
25 waiter 1 dining
26 table 1 dining
27 bar staff 1 dining
28 times 1 time
29 entertainment 1 entertainment
30 boat 1 ship
31 mistake 1 overall
32 big mistake 1 overall
33 negative reviews 0 overall
34 biggest disappointment 0 overall
35 disappointment 0 overall
36 ice cream 1 food
37 bad weather 0 overall
38 food quality 1 food
39 waiters 1 dining
40 buffet restaurant 1 dining
41 night 1 entertainment
42 bad food 1 food
43 main dining room 1 dining
44 complete waste 0 None
45 

C:\Users\fabio\Anaconda\lib\site-packages\IPython\kernel\__main__.py:516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\fabio\Anaconda\lib\site-packages\IPython\kernel\__main__.py:519: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


ValueError: Must pass DataFrame with boolean values only